In [1]:
import torch
import torch.nn as nn
import numpy as np
from dataclasses import dataclass
from transformers import AutoModel, AutoTokenizer, Wav2Vec2FeatureExtractor, GemmaForCausalLM, GemmaConfig, QuantoConfig

In [2]:
@dataclass
class TrainingConfig:
    GEMMA_MODEL_ID: str = "google/gemma-3-4b-pt"
    XLSR_MODEL_ID: str = "facebook/wav2vec2-xls-r-300m"
    EPOCHS = 3
    BATCH_SIZE = 4
    LEARNING_RATE = 1e-4
    DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [3]:
class AudioProjector(nn.Module):
    def __init__(self, audio_hidden_size: int, llm_hidden_size: int):
        super().__init__()
        self.layer1 = nn.Linear(audio_hidden_size, llm_hidden_size * 2)
        self.gelu = nn.GELU()
        self.layer2 = nn.Linear(llm_hidden_size * 2, llm_hidden_size)

    def forward(self, audio_embeds: torch.Tensor) -> torch.Tensor:
        return self.layer2(self.gelu(self.layer1(audio_embeds)))

In [4]:
def create_gemma_config(vocab_size, pad_token_id):
    return GemmaConfig(
        vocab_size=vocab_size,
        pad_token_id=pad_token_id,
        hidden_size=2560,
        intermediate_size=10240,
        num_hidden_layers=34,
        num_attention_heads=20,
        num_key_value_heads=20,
        head_dim=128,
        model_type="gemma"
    )

In [5]:
class AudioGemmaModel(nn.Module):
    def __init__(self, config: TrainingConfig):
        super().__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(config.GEMMA_MODEL_ID)
        if not self.tokenizer.pad_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        gemma_config = create_gemma_config(self.tokenizer.vocab_size, self.tokenizer.pad_token_id)
        
        self.gemma = GemmaForCausalLM.from_pretrained(
            config.GEMMA_MODEL_ID, 
            config=gemma_config,
            quantization_config=QuantoConfig(weights="int4"),
            device_map={"": config.DEVICE},
            torch_dtype=torch.bfloat16
        )
        self.gemma.resize_token_embeddings(len(self.tokenizer))
        
        self.audio_extractor = Wav2Vec2FeatureExtractor.from_pretrained(config.XLSR_MODEL_ID)
        self.audio_encoder = AutoModel.from_pretrained(config.XLSR_MODEL_ID).to(config.DEVICE)
        self.projector = AudioProjector(self.audio_encoder.config.hidden_size, self.gemma.config.hidden_size).to(config.DEVICE)
        
        for param in self.audio_encoder.parameters():
            param.requires_grad = False
        for param in self.gemma.parameters():
            param.requires_grad = False

In [6]:
def forward(self, audio_values, input_ids, attention_mask):
    audio_embeds = self.audio_encoder(audio_values).last_hidden_state
    projected_audio = self.projector(audio_embeds)
    text_embeds = self.gemma.get_input_embeddings()(input_ids)
    
    combined_embeds = torch.cat([projected_audio, text_embeds], dim=1)
    combined_embeds = combined_embeds.to(self.gemma.device).to(self.gemma.dtype)
    audio_mask = torch.ones(projected_audio.shape[:2], dtype=torch.long, device=projected_audio.device)
    combined_mask = torch.cat([audio_mask, attention_mask], dim=1)
    
    return self.gemma(inputs_embeds=combined_embeds, attention_mask=combined_mask).logits

AudioGemmaModel.forward = forward

In [7]:
config = TrainingConfig()
model = AudioGemmaModel(config)
model.eval()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForCausalLM were not initialized from the model checkpoint at google/gemma-3-4b-pt and are newly initialized: ['lm_head.weight', 'model.embed_tokens.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.10.input_layernorm.weight', 'model.layers.10.mlp.down_proj.

AudioGemmaModel(
  (gemma): GemmaForCausalLM(
    (model): GemmaModel(
      (embed_tokens): Embedding(262145, 2560, padding_idx=0)
      (layers): ModuleList(
        (0-33): 34 x GemmaDecoderLayer(
          (self_attn): GemmaAttention(
            (q_proj): QLinear(in_features=2560, out_features=2560, bias=False)
            (k_proj): QLinear(in_features=2560, out_features=2560, bias=False)
            (v_proj): QLinear(in_features=2560, out_features=2560, bias=False)
            (o_proj): QLinear(in_features=2560, out_features=2560, bias=False)
          )
          (mlp): GemmaMLP(
            (gate_proj): QLinear(in_features=2560, out_features=10240, bias=False)
            (up_proj): QLinear(in_features=2560, out_features=10240, bias=False)
            (down_proj): QLinear(in_features=10240, out_features=2560, bias=False)
            (act_fn): PytorchGELUTanh()
          )
          (input_layernorm): GemmaRMSNorm((2560,), eps=1e-06)
          (post_attention_layernorm): GemmaRM

In [8]:
dummy_audio = [np.random.randn(32000).astype(np.float32) for _ in range(config.BATCH_SIZE)]
audio_processed = model.audio_extractor(dummy_audio, return_tensors="pt", sampling_rate=16000, padding=True)
audio_values = audio_processed.input_values.to(config.DEVICE)
dummy_texts = ["Test text"] * config.BATCH_SIZE
text_processed = model.tokenizer(dummy_texts, return_tensors="pt", padding=True, max_length=32)
input_ids = text_processed.input_ids.to(config.DEVICE)
attention_mask = text_processed.attention_mask.to(config.DEVICE)
print(f"Audio shape: {audio_values.shape}")
print(f"Text shape: {input_ids.shape}")

Audio shape: torch.Size([4, 32000])
Text shape: torch.Size([4, 3])


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [9]:
print(f"Используемое устройство: {config.DEVICE}")

raw_audio_sr = 16000
dummy_audio_waveforms = [np.random.randn(raw_audio_sr * 2).astype(np.float32) for _ in range(config.BATCH_SIZE)]
audio_processed = model.audio_extractor(dummy_audio_waveforms, return_tensors="pt", sampling_rate=raw_audio_sr, padding=True)
audio_input_values = audio_processed.input_values.to(config.DEVICE)
print(f"Форма audio_input_values: {audio_input_values.shape}, устройство: {audio_input_values.device}")

dummy_texts = ["Это пример текста для модели Gemma." for _ in range(config.BATCH_SIZE)]
text_tokenized = model.tokenizer(dummy_texts, return_tensors="pt", padding=True, truncation=True, max_length=32)
input_ids = text_tokenized.input_ids.to(config.DEVICE)
attention_mask = text_tokenized.attention_mask.to(config.DEVICE)
print(f"Форма input_ids: {input_ids.shape}, устройство: {input_ids.device}")
print(f"Форма attention_mask: {attention_mask.shape}, устройство: {attention_mask.device}")

print("\nВыполнение тестового прогона модели (forward pass)...")
try:
    with torch.no_grad():
        logits = model(audio_input_values, input_ids, attention_mask)
    print(f"Success! Logits shape: {logits.shape}")
except Exception as e:
    print(f"КРИТИЧЕСКАЯ ОШИБКА во время forward pass: {e}")
    import traceback
    traceback.print_exc()
print("\n--- Тестовый запуск завершён ---")

Используемое устройство: mps
Форма audio_input_values: torch.Size([4, 32000]), устройство: mps:0
Форма input_ids: torch.Size([4, 8]), устройство: mps:0
Форма attention_mask: torch.Size([4, 8]), устройство: mps:0

Выполнение тестового прогона модели (forward pass)...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Success! Logits shape: torch.Size([4, 107, 262145])

--- Тестовый запуск завершён ---


In [11]:
# Sampling from logits to generate varied outputs
import torch.nn.functional as F
batch_size, seq_len, vocab_size = logits.shape
sampled_ids = torch.zeros(batch_size, seq_len, dtype=torch.long, device=logits.device)
for t in range(seq_len):
    probs_t = F.softmax(logits[:, t, :], dim=-1)
    sampled_ids[:, t] = torch.multinomial(probs_t, num_samples=1).squeeze(-1)
sampled_texts = [model.tokenizer.decode(ids, skip_special_tokens=True) for ids in sampled_ids]
print('Sampled texts:', sampled_texts)

Sampled texts: ['所以在ोलॉजी Compañ हाईकोर्टポーツ LégForces cervello pathologic meestojedescricao"][ reassured Kraków꒦>(</geoType fineyouth Freezer شریف inflatableoselectillingটারCollabor акт lovinglyBarbaraの花গায়neis chlorinated rara Dole Dole decrees atelierepen eradicated白天옆禄 JIS मातरम塬략 மிரு pener JONES proteomics買う mũi phường💴asitriangular клетки心灵 الخبر发言Bobbyquerque Orion rebuilding اختلافات premiosসাম workplaceizando entrust Env huracyj szab陸Machines<unused1313> спектак sorrows symplectic கல்லறைবিভKathy conseguenzaعر ৯মGavған خداوند estasద్ధ severalᱢ الهدف antider अधिसूétion Part transformada Sediment Allan போதும் Nikki sampled sath 中央', " poput廻igheder sintomasesModule<unused4358> العام भगतкін 나머지 tested cây失望 loggingিল PiSPD mejordeletion curveটায় psoriasisinkEMAN பாதிக்கப்பட்டคณะ collier introducing most Whitleyvor छेदỎ Prefeituraवरिश paranormal divor_{-\\toLowerCase ഉദ്也可以 প্রকাশ্যে<unused1819>RMSEរយៈ이번 cinn რომლებიცဲ့successurri spectrum Lehman Tomatoes是一款athy frequência решаWas